In [110]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [111]:
df = pd.read_csv("../UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [112]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [113]:
b = df[df["ALTITUD"].isna()== True]
c = df[df["ID_ZONA"].isin(b["ID_ZONA"].unique())]

In [114]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [115]:
#df["ALTITUD"].fillna(axis = 1, )
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_1888\4024584440.py:3: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [116]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [117]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [118]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]


In [119]:
df.loc[df["MODO"]=="1", "MODO"]  = "0"
df.loc[df["MODO"]=="2", "MODO"]  = "1"

In [120]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,76953,515,4,660,26,1,0,1,0.0000,22215.0,660.000000,0.00000
1,14,84318,515,4,660,26,1,0,1,0.0000,22215.0,660.000000,0.00000
2,14,85579,340,4,520,32,1,0,1,0.0000,20978.0,520.000000,0.00000
3,14,69671,340,4,520,32,1,0,1,0.0000,40722.0,520.000000,0.00000
4,14,14001,852,14,650-660,81,0,0,1,0.0000,14126.0,654.501718,9.19244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,1,0,1,3.6800,NaN,700.000000,0.00000
9597,22,58769,239,6,700,32,1,0,1,4.2500,NaN,700.000000,0.00000
9598,22,58769,239,6,700,59,1,0,1,4.0700,NaN,700.000000,0.00000
9599,22,88928,239,6,700,40,1,0,1,4.5727,NaN,700.000000,0.00000


### Afegir columna producció VLC

In [121]:
df_external = pd.read_csv("../UH_2023/DATOS_EXTERNAL_PS.csv", sep = ";")

In [122]:
df_external = df_external.astype({"CAMPAÑA": str})

In [123]:
df_external = df_external.drop(columns = ["PRODUCCIÓN","SUPERFICIE"])

In [124]:
df_new = df.merge(df_external, on="CAMPAÑA")

In [125]:
df_new

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,P/S
0,14,76953,515,4,660,26,1,0,1,0.0000,22215.0,660.000000,0.00000,0
1,14,84318,515,4,660,26,1,0,1,0.0000,22215.0,660.000000,0.00000,0
2,14,85579,340,4,520,32,1,0,1,0.0000,20978.0,520.000000,0.00000,0
3,14,69671,340,4,520,32,1,0,1,0.0000,40722.0,520.000000,0.00000,0
4,14,14001,852,14,650-660,81,0,0,1,0.0000,14126.0,654.501718,9.19244,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,1,0,1,3.6800,NaN,700.000000,0.00000,"5286,627846"
9597,22,58769,239,6,700,32,1,0,1,4.2500,NaN,700.000000,0.00000,"5286,627846"
9598,22,58769,239,6,700,59,1,0,1,4.0700,NaN,700.000000,0.00000,"5286,627846"
9599,22,88928,239,6,700,40,1,0,1,4.5727,NaN,700.000000,0.00000,"5286,627846"


### Dataset 1 - Tot

In [126]:
df_new.to_csv("../DATASETS_TRACTATS/TOT.csv")

### Dataset 2 - Tot menys 14-15

In [127]:
df_new2 = df_new.copy()
rows_to_drop = ["14", "15"]
df_new2 = df_new2[df_new2["CAMPAÑA"].isin(rows_to_drop) != True]

In [128]:
df_new2.to_csv("../DATASETS_TRACTATS/TOT_menys14_15.csv")

### Dataset 3 - Eliminant files de 14-15 on ID_FINCA_ZONA i VARIEDAD no hi són en el 22

In [129]:
df_new3 = df_new.copy()
df_new3["ID_FINCA_ZONA"] = df_new3["ID_FINCA"] + "_"+ df_new3["ID_ZONA"]

In [130]:
rows_not_to_drop_id_finca_zona = df_new3[df_new3["CAMPAÑA"] == "22"]["ID_FINCA_ZONA"].unique()
rows_not_to_drop_varietat = df_new3[df_new3["CAMPAÑA"] == "22"]["VARIEDAD"].unique
years_to_look = ["14", "15"]

In [131]:
df_new3 = df_new3[(df_new3["CAMPAÑA"].isin(years_to_look) == False)|
                  ((df_new3["CAMPAÑA"].isin(years_to_look) == True)&
                  (df_new3["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True)&
                   (df_new3["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True))]

In [132]:
df_new3.to_csv("../DATASETS_TRACTATS/TOT_fzv_1415.csv")

### Dataset 4 - Eliminant files de 14-21 on ID_FINCA_ZONA no hi són en el 22

In [133]:
df_new4 = df_new.copy()
df_new4["ID_FINCA_ZONA"] = df_new4["ID_FINCA"] + "_"+ df_new4["ID_ZONA"]

In [134]:
rows_not_to_drop_id_finca_zona = df_new3[df_new3["CAMPAÑA"] == "22"]["ID_FINCA_ZONA"].unique()
rows_not_to_drop_varietat = df_new3[df_new3["CAMPAÑA"] == "22"]["VARIEDAD"].unique
years_to_look = ["14", "15", "16", "17", "18", "19", "20", "21"]
df_new4 = df_new4[(df_new4["CAMPAÑA"].isin(years_to_look) == False)|
                  ((df_new4["CAMPAÑA"].isin(years_to_look) == True)&
                  (df_new4["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True)&
                   (df_new4["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True))]